In [9]:
!wget http://www.rondhuit.com/download/ldcc-20140209.tar.gz
!tar xfz ldcc-20140209.tar.gz

category2id = {"kaden-channel":0, "peachy":1, "sports-watch":2, "dokujo-tsushin":3, "livedoor-homme":4, "it-life-hack":5, "movie-enter":6}

# Livedoorニュースコーパスからテキスト分類用のデータを作成．
import glob
import random

# 1行は [文][TAB][ラベル]からなる
write_lines = []
for d in category2id.keys():
  for file in glob.glob("text/" + d + "/*.txt"):
    with open(file) as f:
      lines = f.readlines()
      # 最初の2行はURLと日付なので捨てる
      for line in lines[3:]:
        line = line.strip()
        if len(line) > 20 and len(line) < 256 and "http" not in line:
          write_lines.append(line + "\t" + str(category2id[d]) + "\n")

random.shuffle(write_lines)
# Train, Dev, Testの3つに分ける.
dev = write_lines[0:2000]
test =write_lines[2000:4000]
train = write_lines[4000:]
# ファイルに保存
w = open("train.tsv", "w")
w.writelines(train)
w.close()
w = open("dev.tsv", "w")
w.writelines(dev)
w.close()
w = open("test.tsv", "w")
w.writelines(test)
w.close()
print("data size ", len(train), len(dev), len(test))

!head dev.tsv

#83.pyが使える

--2020-08-22 15:09:13--  http://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.rondhuit.com/download/ldcc-20140209.tar.gz [following]
--2020-08-22 15:09:13--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz.3’

ldcc-20140209.tar.g 100%[===================>]   8.44M  8.32MB/s    in 1.0s    

2020-08-22 15:09:14 (8.32 MB/s) - ‘ldcc-20140209.tar.gz.3’ saved [8855190/8855190]

data size  67653 2000 2000
■新成人女性の3人に1人は、恋人選びで"車"もチェック	1
オシャレアイテムのひとつでもあるサングラス。実は目を守ることは紫外線対策にはすごく大事なもので、冒頭で触れた目の下のクマや目尻のシワの原因になっているほか、将来的には白内障にかかるリスクも増えるのだとか。また、髪も長時間紫外線を浴びていると、変色し

In [11]:
!pip install transformers
!apt install git make curl xz-utils file
!apt install mecab libmecab-dev mecab-ipadic mecab-ipadic-utf8
!pip install mecab-python3==0.996.5

import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM
import transformers

pretrained_model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'

# 事前学習済みモデルのトークナイザを使用
tokenizer = BertJapaneseTokenizer.from_pretrained(pretrained_model_name)
#中でめかぶを使っている
# 形態素解析 (内部でMeCabを使用)
text1 = "今日はいい天気だね"
text2 = "明日は雨がふるかもしれませんね"

print("text1", tokenizer.tokenize(text1))
print("text2", tokenizer.tokenize(text2))

"""
text1 ['今日', 'は', 'いい', '天気', 'だ', 'ね']
text2 ['明日', 'は', '雨', 'が', 'ふる', 'かも', 'しれ', 'ませ', 'ん', 'ね']
"""

# BERTに入力する形式に変換　
for_bert_inputs = tokenizer([text1, text2], padding=True, return_tensors="pt")
print("for_bert_inputs", for_bert_inputs)
"""
for_bert_inputs {'input_ids': tensor([[    2,  3246,     9,  2575, 11385,    75,  1852,     3,     0,     0,
             0,     0],
        [    2, 11475,     9,  3741,    14,  8491,  4830,  6758,  6769,  1058,
          1852,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
"""
# input_ids: 単語をIDに変換した系列．padding済み
# token_type_ids： 2文からなるペアを入力した場合に，1文目と2文目を区別するための系列
# attention_mask： input_idsのpadding部分とそうでない部分を区別するための系列


# 参考．文のペアを入れる場合の例
text3 = "そうかな"
text4 = "違うと思います"
tmp = tokenizer([[text1, text2],[text3, text4]], padding=True, return_tensors="pt")
print("tmp", tmp)
"""
tmp {'input_ids': tensor([[    2,  3246,     9,  2575, 11385,    75,  1852,     3, 11475,     9,
          3741,    14,  8491,  4830,  6758,  6769,  1058,  1852,     3],
        [    2,  1778,    29,    18,     3,  5720,    13,  2502,  2610,     3,
             0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}
"""


# BERTの事前学習済みモデルをロード．BertForSequenceClassificationは1文が与えられて分類を行うクラス．num_labelsでラベル数を指定
# 他にもいろいろ用意されている．使用例も書かれている→  https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
model = transformers.BertForSequenceClassification.from_pretrained(pretrained_model_name, num_labels=5)

out = model(input_ids=for_bert_inputs["input_ids"], token_type_ids=for_bert_inputs["token_type_ids"], attention_mask=for_bert_inputs["attention_mask"])
print(for_bert_inputs["input_ids"].shape)
print("out", out)

model = transformers.BertModel.from_pretrained(pretrained_model_name)
out = model(input_ids=for_bert_inputs["input_ids"], token_type_ids=for_bert_inputs["token_type_ids"], attention_mask=for_bert_inputs["attention_mask"])
# pooled_output = outputs[1]

# pooled_output = self.dropout(pooled_output)
# logits = self.classifier(pooled_output)
print("out", out[1].size())
"""
out (tensor([[-0.4694, -0.2888,  0.1584,  0.1443,  0.2474],
        [-0.4795, -0.2614,  0.0915,  0.1614,  0.1209]],
       grad_fn=<AddmmBackward>),)
"""
#線形層に通したものがでてくる
# あとは損失を計算していつもどおりbackpropすればOK

Reading package lists... Done
Building dependency tree       
Reading state information... Done
make is already the newest version (4.1-9.1ubuntu1).
xz-utils is already the newest version (5.2.2-1.3).
curl is already the newest version (7.58.0-2ubuntu3.9).
file is already the newest version (1:5.32-2ubuntu0.4).
git is already the newest version (1:2.17.1-1ubuntu0.7).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libmecab-dev is already the newest version (0.996-5).
mecab is already the newest version (0.996-5).
mecab-ipadic is already the newest version (2.7.0-20070801+main-1).
mecab-ipadic-utf8 is already the newest version (2.7.0-20070801+main-1).
The following package was automatically installed and is no longer required:
  libnvidia-comm

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

torch.Size([2, 12])
out (tensor([[-0.3102,  0.1310,  0.1940, -0.1165, -0.0578],
        [-0.2855,  0.2153,  0.1958, -0.0514, -0.0847]],
       grad_fn=<AddmmBackward>),)
out torch.Size([2, 768])


'\nout (tensor([[-0.4694, -0.2888,  0.1584,  0.1443,  0.2474],\n        [-0.4795, -0.2614,  0.0915,  0.1614,  0.1209]],\n       grad_fn=<AddmmBackward>),)\n'

In [12]:
import pickle
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_sequence, pad_packed_sequence, pad_sequence, pack_padded_sequence
!pip install transformers
!apt install git make curl xz-utils file
!apt install mecab libmecab-dev mecab-ipadic mecab-ipadic-utf8
!pip install mecab-python3==0.996.5

#import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM
import transformers

pretrained_model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'

# 事前学習済みモデルのトークナイザを使用
tokenizer = BertJapaneseTokenizer.from_pretrained(pretrained_model_name)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
make is already the newest version (4.1-9.1ubuntu1).
xz-utils is already the newest version (5.2.2-1.3).
curl is already the newest version (7.58.0-2ubuntu3.9).
file is already the newest version (1:5.32-2ubuntu0.4).
git is already the newest version (1:2.17.1-1ubuntu0.7).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libmecab-dev is already the newest version (0.996-5).
mecab is already the newest version (0.996-5).
mecab-ipadic is already the newest version (2.7.0-20070801+main-1).
mecab-ipadic-utf8 is already the newest version (2.7.0-20070801+main-1).
The following package was automatically installed and is no longer required:
  libnvidia-comm

In [43]:
import time
def get_data(fname):
    label_list = [] # ラベル(カテゴリーのid)を格納するリスト
    ids_list = [] # タイトルに含まれる単語のID系列のtensorを格納するリスト

    with open(fname) as f:
        for line in f:
            if not line:
                continue
            line = line.strip()
            if len(line.split("\t")[1]) != 1:
                continue
                
            title = line.split("\t")[0] #
            ids_list.append(title) 

            label = line.split("\t")[1] 
            label = int(label) #str2intに
            label_list.append(label) #
    labels = torch.tensor(label_list) #label_listをtensorに変換 
    return ids_list, labels

class RnnDataset(torch.utils.data.Dataset):
    def __init__(self, data, label):
        self.for_bert_inputs = tokenizer(data, padding=True, return_tensors="pt")
        self.label = label

    def __len__(self):
        return len(self.for_bert_inputs["input_ids"])

    def __getitem__(self, idx):
        return self.for_bert_inputs["input_ids"][idx], self.for_bert_inputs["token_type_ids"][idx], self.for_bert_inputs["attention_mask"][idx], self.label[idx]




batch_size = 64

#vocab_size = max(word2id.values()) + 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# データの準備
train_path =  "train.tsv"#学習用ファイルパス
train_x, train_y = get_data(train_path)
valid_path = "dev.tsv"#todo 評価用ファイルパス
valid_x, valid_y = get_data(valid_path)
test_path = "test.tsv"
test_x, test_y = get_data(test_path)


dataset = RnnDataset(train_x, train_y)

train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = transformers.BertForSequenceClassification.from_pretrained(pretrained_model_name, num_labels=7)
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.05)
loss_fn = nn.CrossEntropyLoss()


def train(model, train_loader):
    s = time.time()
    #print(1)
    model.train()
    total_loss = 0
    correct_num = 0
    i = 0
    len_loader = len(train_loader)
    for data, types, mask, target in train_loader:
        # GPUへ
        i=i+batch_size
        print(i)
        #print(data.shape)
        #print(types.shape)
        #print(mask.shape)
        #print(target.shape)
        data = data.to(device)
        types = types.to(device)
        mask = mask.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        pred = model(input_ids=data, token_type_ids=types, attention_mask=mask) #返り値tuple型
        pred = pred[0]
        #print(pred)
        #print(target)
        #pred = torch.tensor(pred)
        #print(pred.type, target.type)
        loss = loss_fn(pred, target) #lossを計算
        loss.backward() #逆伝播
        optimizer.step()
        #print("loss: ", loss)
        total_loss+= loss.item() #lossをtotal_lossに加算
        correct_num += (torch.argmax(pred, axis=1)==target).sum().item() #予測の正解数をカウント
        #print(correct_num)

            
    batch_loss = total_loss/len_loader #バッチごとの平均ロス
    acc = correct_num/len_loader#todo 正解率
    print("time: ", time.time()-s)
    return batch_loss, acc


for epoch in range(30):
    train_loss, train_acc = train(model, train_loader)

    print(f"epoch: {epoch}")
    print(f"<train> Loss: {train_loss}\tAccuracy: {train_acc}")
   # print(f"<valid> Loss: {valid_loss}\tAccuracy: {valid_acc}")

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

64


RuntimeError: ignored

In [25]:
torch.tensor([1, 2, 3])

tensor([1, 2, 3])